In [1]:
library(orthogene)
library(Seurat)
library(dplyr)

Registered S3 methods overwritten by 'treeio':
  method              from    
  MRCA.phylo          tidytree
  MRCA.treedata       tidytree
  Nnode.treedata      tidytree
  Ntip.treedata       tidytree
  ancestor.phylo      tidytree
  ancestor.treedata   tidytree
  child.phylo         tidytree
  child.treedata      tidytree
  full_join.phylo     tidytree
  full_join.treedata  tidytree
  groupClade.phylo    tidytree
  groupClade.treedata tidytree
  groupOTU.phylo      tidytree
  groupOTU.treedata   tidytree
  inner_join.phylo    tidytree
  inner_join.treedata tidytree
  is.rooted.treedata  tidytree
  nodeid.phylo        tidytree
  nodeid.treedata     tidytree
  nodelab.phylo       tidytree
  nodelab.treedata    tidytree
  offspring.phylo     tidytree
  offspring.treedata  tidytree
  parent.phylo        tidytree
  parent.treedata     tidytree
  root.treedata       tidytree
  rootnode.phylo      tidytree
  sibling.phylo       tidytree

The legacy packages maptools, rgdal, and rgeos, under

In [2]:
wd <- '~/codebases/MacBrainDev/'
setwd(wd)

In [3]:
n.cores <- as.numeric(Sys.getenv('SLURM_CPUS_PER_TASK'))
n.cores <- if (is.na(n.cores)) parallel::detectCores() else n.cores

# Cargar los datos

In [8]:
data.dir <- 'data/'

base.name <- 'All.MNN.v1.org.fct'

# Define filename
indata.fname <- paste0(data.dir, base.name, '.rds')
pc.groups.file <- 'raw/Patterning_center_cluster_order.tsv'
disease.lists.file <- paste0(data.dir, 'all_diseases_list.rds')

In [9]:
# Load data
indata <- readRDS(indata.fname)
# Print summary
indata

clusters <- data.frame(unique(indata@meta.data[,c('subclass','subtype')]))
rownames(clusters) <- clusters$subtype

clusters[clusters$subclass == 'Patterning centers',]

pc.groups <- read.table(pc.groups.file, sep='\t', skip = 1, row.names = 1)
pc.groups

An object of class Seurat 
34619 features across 761529 samples within 1 assay 
Active assay: RNA (34619 features, 0 variable features)
 2 dimensional reductions calculated: mnn, umap

,subclass,subtype
,<fct>,<fct>
PC NKX2-1 NKX6-2,Patterning centers,PC NKX2-1 NKX6-2
PC FGF17,Patterning centers,PC FGF17
PC NKX2-1 LMO1,Patterning centers,PC NKX2-1 LMO1
PC SFRP2,Patterning centers,PC SFRP2
PC RSPO3,Patterning centers,PC RSPO3
PC TTR,Patterning centers,PC TTR
PC TCF7L2,Patterning centers,PC TCF7L2
GE RG NKX2-1 OLIG1,Patterning centers,GE RG NKX2-1 OLIG1
GE RG NKX2-1 DLK1,Patterning centers,GE RG NKX2-1 DLK1


,V2,V3,V4
,<int>,<int>,<int>
PC FGF17,1,1,1
PC SFRP1,0,0,1
PC NKX2-1 NKX6-2,1,1,1
PC NKX2-1 LMO1,1,1,1
GE RG NKX2-1 DLK1,1,0,1
GE RG NKX2-1 OLIG1,1,0,1
PC RSPO3,1,1,1
PC TTR,1,1,1
PC SFRP2,1,1,1


In [10]:
# 1. All clusters
g1 <- levels(clusters$subclass)
# 2. All subclusters
g2 <- levels(clusters$subtype)

# 3. Groups de patterning centers, distintas versiones
g3 <- rownames(pc.groups)[pc.groups$V2==1]
g4 <- rownames(pc.groups)[pc.groups$V3==1]
g5 <- rownames(pc.groups)[pc.groups$V4==1]

# 4. Groups de 'Patterning centers' 'dorsal NSC' 'GE NSC'
g6 <- as.character(clusters$subtype[as.character(clusters$subclass)%in%c('Patterning centers', 'dorsal NSC', 'GE NSC')])
g7 <- as.character(clusters$subtype[as.character(clusters$subclass)%in%c('dorsal NSC', 'GE NSC')])

In [11]:
grupos <- list(
    All_clusters=list(group.by='subclass', idents=g1),
    All_subclusters=list(group.by='subtype', idents=g2),
    PCs_1=list(group.by='subtype', idents=g3),
    PCs_2=list(group.by='subtype', idents=g4),
    PCs_3=list(group.by='subtype', idents=g5),
    Progenitors=list(group.by='subtype', idents=g6),
    Neural_Stem_Cells=list(group.by='subtype', idents=g7))

for (gname in names(grupos)){
    grupos[[gname]][['group.name']] <- gname
}

grupos
saveRDS(grupos, file = 'ewce_exp_groups.rds')

$All_clusters
$All_clusters$group.by
[1] "subclass"

$All_clusters$idents
 [1] "Patterning centers"   "dorsal NSC"           "enIPC"               
 [4] "Excitatory neurons"   "CR"                   "GE NSC"              
 [7] "inIPC"                "Inhibitory neurons"   "gIPC"                
[10] "Astro"                "OPC&Oligo"            "Mes"                 
[13] "Immune"               "RB&Vas"               "PAT-related subtypes"

$All_clusters$group.name
[1] "All_clusters"


$All_subclusters
$All_subclusters$group.by
[1] "subtype"

$All_subclusters$idents
  [1] "PC FGF17"               "PC SFRP1"               "PC NKX2-1 LMO1"        
  [4] "PC NKX2-1 NKX6-2"       "GE RG NKX2-1 DLK1"      "GE RG NKX2-1 OLIG1"    
  [7] "PC NKX2-1 RAX"          "PC RSPO3"               "PC TTR"                
 [10] "PC SFRP2"               "PC TCF7L2"              "NESC RSPO3 DIRAS3"     
 [13] "NESC RSPO3 TEX15"       "vRG HMGA2 CCND1"        "RGC FABP7 PMP22"       
 [16] "vRG SAT1 STMN2"         "tRG CRYAB MEST"         "tRG CRYAB FNDC1"       
 [19] "oRG HOPX TNC"           "oRG HOPX APOE"          "Ependymal FOXJ1"       
 [22] "IPC EOMES VIM"          "IPC EOMES NEUROG1"      "IPC EOMES NHLH1 deep"  
 [25] "IPC EOMES NHLH1 up"     "ExN SOX5 PALMD"         "ExN SOX5 ID2"          
 [28] "ExN SOX5 NR4A2 GRID2"   "ExN SOX5 SYT6"          "ExN SOX5 KCNV1"        
 [31] "ExN SOX5 OPRK1 NR4A2"   "ExN SOX5 OPRK1 SULF1"   "ExN CUX2 PALMD"        
 [34] "ExN CUX2 ADRA2A"        "ExN CUX2 ACTN2"         "ExN TSHZ2 NR4A3"       
 [37] "IPC RSPO3 NEUROG1"      "IPC RSPO3 NHLH1"        "CR ETV1"               
 [40] "CR TP73"                "GE CR ZFHX3"            "GE RG HMGA2 NKX6-2"    
 [43] "GE RG HMGA2 NR2F2"      "GE RG HOPX STMN1"       "GE RG HOPX NRG1"       
 [46] "GE RG STMN2 SOX5"       "GE RG CRYAB"            "GE Ependymal FOXJ1"    
 [49] "inIPC ASCL1 DLX1"       "inIPC ASCL1 SFRP2"      "inIPC ASCL1 GSX1"      
 [52] "inIPC ASCL1 GADD45G"    "inIPC ASCL1 GPX1"       "inIPC ASCL1 SP8"       
 [55] "inIPC ASCL1 NKX2-1"     "inIPC DLX1 NR2F2"       "InN NKX2-1 CCND2"      
 [58] "InN LHX8 ZIC1"          "InN LHX6 CCK"           "InN LHX6 CRABP1"       
 [61] "InN LHX6 MAF"           "InN LHX6 SST RELN"      "InN LHX6 SST NPY"      
 [64] "InN LHX6 DCN"           "InN LHX6 GUCY1A2"       "InN HMX1"              
 [67] "InN NR2F2 CRH"          "InN NR2F2 LAMP5"        "InN NR2F2 VIP"         
 [70] "InN NR2F2 SP8 KIT"      "InN NR2F2 SP8"          "InN SP8 CRH"           
 [73] "InN SP8 VIP"            "InN SP8 MEIS2 RND3"     "InN SP8 PAX6"          
 [76] "InN ZIC1 ZIC2"          "InN GNRH1"              "InN SFRP2 MEIS2"       
 [79] "InN SIX3 ZFHX3"         "InN FOXP1 ISL1"         "InN FOXP1 PENK"        
 [82] "InN FOXP2 TSHZ1"        "gIPC EGFR"              "gIPC EGFR MKI67"       
 [85] "aIPC IGFBP2"            "oIPC DLL1"              "Astro EGFR"            
 [88] "Astro GFAP"             "Astro MFGE8"            "OPC PDGFRA"            
 [91] "OPC PDGFRA MKI67"       "OPC PCDH15 COL9A1"      "Oligo MBP PLP1"        
 [94] "Mesenchymal LUM CYTL1"  "Mesenchymal FOXD3 PLP1" "Microglia PTPRC C1QC"  
 [97] "T PTPRC CD69"           "RB HBA1 HBE1"           "RB HBA1 SNCA"          
[100] "Endothelial CLDN5 FLT1" "Endothelial CLDN5 RGS5" "Pericyte GRM8"         
[103] "SMC ACTA2"              "VLMC PTGDS DSP"         "Cls FGF17 LGI1"        
[106] "Cls LHX9 EBF1"          "Cls RSPO3 SOX1"         "Cls GSX2 B3GAT2"       
[109] "IPC FGF17"              "IPC TCF7L2"             "Neu TAGLN3 NRN1"       
[112] "Neu TAGLN3 ONECUT2"    

$All_subclusters$group.name
[1] "All_subclusters"


$PCs_1
$PCs_1$group.by
[1] "subtype"

$PCs_1$idents
[1] "PC FGF17"           "PC NKX2-1 NKX6-2"   "PC NKX2-1 LMO1"    
[4] "GE RG NKX2-1 DLK1"  "GE RG NKX2-1 OLIG1" "PC RSPO3"          
[7] "PC TTR"             "PC SFRP2"           "PC TCF7L2"         

$PCs_1$group.name
[1] "PCs_1"


$PCs_2
$PCs_2$group.by
[1] "subtype"

$PCs_2$idents
[1] "PC 

In [12]:
## 2. INDATA (mmulatta) -> HSDATA (hsapiens)

exp.ort <- convert_orthologs(rownames(indata), input_species = 'mmulatta', output_species = 'hsapiens', verbose=T)

data.mm.to.hs <- setNames(rownames(exp.ort), exp.ort$input_gene)
data.hs.to.mm <- setNames(exp.ort$input_gene, rownames(exp.ort))

hsdata <- CreateSeuratObject(counts = indata[['RNA']]@counts[names(data.mm.to.hs),], project = 'hs', assay = 'RNA', meta.data = indata@meta.data, 
                             row.names = data.mm.to.hs, min.cells = 0, min.features = 0)
hsdata <- SetAssayData(object = hsdata, slot = 'data', assay = 'RNA', new.data=indata[['RNA']]@data[names(data.mm.to.hs),])

hsdata

Preparing gene_df.

character format detected.

Converting to data.frame

Extracting genes from input_gene.

34,619 genes extracted.

Converting mmulatta ==> hsapiens orthologs using: gprofiler

Retrieving all organisms available in gprofiler.

Using stored `gprofiler_orgs`.

Mapping species name: mmulatta

1 organism identified from search: mmulatta

Retrieving all organisms available in gprofiler.

Using stored `gprofiler_orgs`.

Mapping species name: hsapiens

1 organism identified from search: hsapiens

Checking for genes without orthologs in hsapiens.

Extracting genes from input_gene.

36,866 genes extracted.

Extracting genes from ortholog_gene.

36,866 genes extracted.

Dropping 16,190 NAs of all kinds from ortholog_gene.

Checking for genes without 1:1 orthologs.

Dropping 1,172 genes that have multiple input_gene per ortholog_gene (many:1).

Dropping 1,560 genes that have multiple ortholog_gene per input_gene (1:many).

Filtering gene_df with gene_map

Setting ortholog_gene t

An object of class Seurat 
17330 features across 761529 samples within 1 assay 
Active assay: RNA (17330 features, 0 variable features)

In [13]:
## 3. Genes de enfermedad presentes en los datos

disease_lists <- readRDS(disease.lists.file)
disease_genes <- unique(unlist(disease_lists))

disease_lists.data <- lapply(disease_lists, function(x){x[x%in%rownames(hsdata)]})
disease_genes.data <- unique(unlist(disease_lists.data))

important.gene.sets <- list(
    disease_lists=disease_lists,
    disease_lists.data=disease_lists.data,
    disease_genes=disease_genes,
    disease_genes.data=disease_genes.data,
    data.mm.to.hs=data.mm.to.hs,
    data.hs.to.mm=data.hs.to.mm,
    hs.data.genes=rownames(hsdata)
)

saveRDS(important.gene.sets, file = 'data/ewce_important_genesets.rds')

## 4. Retrieve data

In [24]:
### Subset data from groups
subset_grupo <- function(sobj, grupo){
    
    if (grupo$group.by == 'subtype'){
        subseurat <- subset(sobj, subset= subtype %in% grupo$idents)
    } else if (grupo$group.by == 'subclass'){
        subseurat <- subset(sobj, subset= subclass %in% grupo$idents)
    }
    Idents(subseurat) <- grupo$group.by
    return(subseurat)
}


### FindAllMarkers
markers_subdata <- function(subdata, 
                            features = NULL, 
                            max.cells.per.ident = 1000){
    
    markers <- FindAllMarkers(
        subdata,
        assay = 'RNA',
        features = features,
        logfc.threshold = 0.25,
        test.use = "wilcox",
        slot = "data",
        min.pct = 0.1,
        min.diff.pct = -Inf,
        node = NULL,
        verbose = TRUE,
        only.pos = TRUE,
        max.cells.per.ident = max.cells.per.ident,
        random.seed = 1,
        latent.vars = NULL,
        min.cells.feature = 3,
        min.cells.group = 3,
        pseudocount.use = 1,
        mean.fxn = NULL,
        fc.name = NULL,
        base = 2,
        return.thresh = 0.01,
        densify = FALSE,
        )
    return(markers)
    }

### Scale data using disease-risk & marker genes
scale_subdata <- function(subdata, 
                          features=NULL){
    
    print(paste('Scaling', length(features), 'genes'))
    
    subdata <- ScaleData(subdata,
                         assay = 'RNA',
                         features=features,
                         vars.to.regress = 'nCount_RNA',
                         split.by = NULL,
                         model.use = "linear",
                         use.umi = FALSE,
                         do.scale = TRUE,
                         do.center = TRUE,
                         scale.max = 1E6,
                         min.cells.to.block = 3000,
                         verbose = TRUE
            )
    return(subdata)
    }


### DotPlot data using disease genes
dp_subdata <- function(subdata, glists=disease_lists.data){
    
    dp.data <- do.call('rbind',
                       lapply(names(glists), 
                              function(dis){
                                  d <- DotPlot(subdata, features=glists[[dis]])$data
                                  d$Disease.listname <- dis
                                  d
                              }))
    return(dp.data)
}
    

### Get data meta-function
get_exp_data <- function(grupo, sobj=hsdata, cat=F
                        ){
    
    if (cat){
        
        report.fname <- paste0(data.dir, 'Disease_expression_data.Report.', grupo$group.name, '.txt')
        return(capture.output(get_exp_data(grupo=grupo, sobj=sobj, data.dir=data.dir, cat=F),
                              file = report.fname))
        
    }
    dir.create(data.dir)
    
    print(paste('INIT:', grupo$group.name))

    markers.fname <- paste0(data.dir, 'Disease_expression_data.Markers.', grupo$group.name, '.csv')    
    dpdata.fname <- paste0(data.dir, 'Disease_expression_data.DotPlot.', grupo$group.name, '.csv')
    scaled.fname <- paste0(data.dir, 'Disease_expression_data.ScaleData.', grupo$group.name, '.rds')
        
    if (all(sapply(c(markers.fname, dpdata.fname), file.exists))){
        print('done')
        return()
    }

    if (!all(sapply(c(dpdata.fname), file.exists))){
        if (file.exists(scaled.fname)){
            mode <- "load.rds"
        } else {
            mode <- 'ss+sc'
        }
    } else if (file.exists(markers.fname)){
        return()
    } else {
        mode <- 'ss'
    }
    
    print(mode)
    
    if (mode == 'load.rds'){
        print('Init load data')
        subdata <- readRDS(scaled.fname)
        print('Done load data')
    } else {
        print('Init subset data')
        subdata <- subset_grupo(sobj, grupo)
        print('Done subset data')
    }
    
    if (!file.exists(markers.fname)){
        print('Init markers')
        markers <- markers_subdata(subdata, max.cells.per.ident = Inf)
        write.csv(markers, markers.fname, row.names=T)
    } else {
        markers <- read.csv(markers.fname, row.names=1)
    }
    scale.genes <- union(markers$gene, disease_genes.data)
    print('Done markers')

    if (mode == 'ss+sc'){
        print('Init scale data')
        subdata <- scale_subdata(subdata, features=scale.genes)
        dim(subdata[['RNA']]@scale.data)
        
        if (!file.exists(scaled.fname)){
            print('Init save scale.data')
            saveRDS(object=subdata[['RNA']]@scale.data, file = scaled.fname)
        }
        print('Done save scale.data')
    }
    print('Done scale data')
    
    if (!file.exists(dpdata.fname)){
        print('Init dot plot data')
        dpdata <- dp_subdata(subdata)
        write.csv(dpdata, dpdata.fname, row.names=T)
    }
    print('Done dot plot data')


    print(paste('DONE:', grupo$group.name))
    return(NULL)
}


## 5. Retrieve data for all groups
### Set markers for All_subgroups from markers used for EWCE

In [25]:
grupo <- grupos[['All_subclusters']]

In [26]:
subcs.markers.fname <- paste0(data.dir, 'ewce_downsampled_markers.All.MNN.v1.org.fct.dsInf.rds')
subcs.markers <- readRDS(subcs.markers.fname)
subcs.markers$gene <- data.mm.to.hs[subcs.markers$gene]
subcs.markers <- subset(subcs.markers, !is.na(gene))

In [27]:
markers.fname <- paste0( 'data/Disease_expression_data.Markers.', grupo$group.name, '.csv')    
markers.fname

write.csv(subcs.markers, markers.fname, row.names=T)

[1] "data/Disease_expression_data.Markers.All_subclusters.csv"

### Run all

In [28]:
grupos <- grupos[c('PCs_1', 'All_subclusters')]

In [29]:
library(parallel)

r <- mclapply(grupos, get_exp_data, mc.cores = length(grupos))

r

$PCs_1
NULL

$All_subclusters
NULL